Стоит задача по увеличению LTV за счет уменьшения оттока пользователей и увеличения частоты контакта. 
 
Необходимо провести исследование и ответить на вопрос, каким способом можно достичь указанной цели.

Нужно сделать RFM-анализ и подготовить рекомендации команде сервиса. 

In [1]:
import pandas as pd
import seaborn as sns
from datetime import timedelta

Дан tsv файл запакованный в архив.

Колонки файла представлены:

timestamp хита 

user_id

url - url хита или "click", если пользователь перешел в интернет-магазин 

geo_id - идентификатор региона пользователя 

category - идентификатор категории 

price - стоимость клика, если пользователь перешел в интернет-магазин

In [2]:
rfm = pd.read_csv('rfm.tar.gz', compression = 'gzip', encoding = 'unicode_escape', sep = '\t',
                  header = 0, names = ['timestamp', 'user_id', 'URL_click', 'geo_id', 'category_id', 'price'], 
                  usecols = [0, 1, 2, 3, 4, 5])

C:\Users\User\AppData\Local\Temp\ipykernel_14652\1411565926.py:1: DtypeWarning: Columns (3,4) have mixed types. Specify dtype option on import or set low_memory=False.
  rfm = pd.read_csv('rfm.tar.gz', compression = 'gzip', encoding = 'unicode_escape', sep = '\t',


In [3]:
rfm.head()

,timestamp,user_id,URL_click,geo_id,category_id,price
0,1.375704e+09,00003cb9076039f584a12c2036fd3602,/gate/cart-list.xml?sk=y5fc6079b758f5a4e3646a8...,10000,NaN,NaN
1,1.374480e+09,00003cb9076039f584a12c2036fd3602,/gate/cart-list.xml?sk=yd6558780959d5130b32e1a...,10000,NaN,NaN
2,1.374480e+09,00003cb9076039f584a12c2036fd3602,/gate/cart-list.xml?sk=yd6558780959d5130b32e1a...,10000,NaN,NaN
3,1.374480e+09,00003cb9076039f584a12c2036fd3602,/gate/cart-list.xml?sk=yd6558780959d5130b32e1a...,10000,NaN,NaN
4,1.374480e+09,00003cb9076039f584a12c2036fd3602,/gate/cart-list.xml?sk=yd6558780959d5130b32e1a...,10000,NaN,NaN


In [4]:
rfm['Date'] = pd.to_datetime(rfm.timestamp, unit = 's')

In [5]:
rfm.Date.max()

Timestamp('2013-12-31 19:58:37')

Последняя дата указана в декабре 2013 года.

Текущий год 2022, чтобы небыло огромных значений давности перехода в recency колонке, расчетным днем будет следующий, после последнего перехода.

In [6]:
nw = rfm.Date.max() + timedelta(days = 1)

In [7]:
rfm.tail()

,timestamp,user_id,URL_click,geo_id,category_id,price,Date
10007142,1.361890e+09,fffffdbc5c7fbe34b444d9c213bba286,/offers.xml?modelid=8464553&hid=4165204&how=ap...,213.0,4165204,NaN,2013-02-26 14:42:55
10007143,1.361890e+09,fffffdbc5c7fbe34b444d9c213bba286,/offers.xml?modelid=981213&hid=294661&hyperid=...,213.0,294661,NaN,2013-02-26 14:41:14
10007144,1.361889e+09,fffffdbc5c7fbe34b444d9c213bba286,/offers.xml?modelid=981770&hid=91033&how=aprice,213.0,91033,NaN,2013-02-26 14:36:03
10007145,1.361889e+09,fffffdbc5c7fbe34b444d9c213bba286,/offers.xml?modelid=981770&hid=91033&how=aprice,213.0,91033,NaN,2013-02-26 14:36:07
10007146,NaN,NaN,NaN,NaN,NaN,NaN,NaT


In [8]:
rfm.drop(rfm[rfm.user_id.isna()].index, inplace = True)

Для входа в категорию "Пользователь" буду считать хотябы 1 переход.

Проверяю, есть ли пропущиенные значения в колонке с click хитами.

In [9]:
rfm[~rfm.price.isna()].price.count()

363313

In [10]:
rfm[rfm.URL_click == 'click'].price.count()

363313

In [34]:
rfm[rfm.URL_click != 'click'].price.unique()

array([nan])

In [11]:
rfm_stats = rfm[rfm.URL_click == 'click'].groupby('user_id').agg({'Date' : lambda x: (nw - x.max()).days,
                                                     'URL_click' : 'count',
                                                     'price' : 'sum'})

In [12]:
rfm_stats.head()

,Date,URL_click,price
user_id,,,
00023fcc5c93cb4674bdad0d448272e6,70,25,425.0
00029bdb868e12bb62b8c574b274dbb7,363,5,90.0
00035d94fcd7e814d56531cf95b81ceb,150,3,159.0
00045ac92fb4ed78bc66c40f52775acd,59,5,50.0
00056ac58d63ef1d4b5fb73324bf845d,93,11,145.0


In [13]:
rfm_stats.rename(columns = {'Date' : 'recency', 'URL_click' : 'frequency', 'price' : 'monetary'}, inplace = True)

In [14]:
rfm_stats.head()

,recency,frequency,monetary
user_id,,,
00023fcc5c93cb4674bdad0d448272e6,70,25,425.0
00029bdb868e12bb62b8c574b274dbb7,363,5,90.0
00035d94fcd7e814d56531cf95b81ceb,150,3,159.0
00045ac92fb4ed78bc66c40f52775acd,59,5,50.0
00056ac58d63ef1d4b5fb73324bf845d,93,11,145.0


In [15]:
rfm_stats.describe()

,recency,frequency,monetary
count,55277.000000,55277.000000,55277.000000
mean,155.282161,6.572589,185.546719
std,112.010076,20.733833,572.142003
min,1.000000,1.000000,0.000000
25%,49.000000,1.000000,20.000000
50%,141.000000,2.000000,50.000000
75%,256.000000,5.000000,154.000000
max,365.000000,2356.000000,37020.000000


In [35]:
rfm_stats.describe(percentiles=[0.2, 0.5, 0.8, 0.96])

,recency,frequency,monetary
count,55277.000000,55277.000000,55277.000000
mean,155.282161,6.572589,185.546719
std,112.010076,20.733833,572.142003
min,1.000000,1.000000,0.000000
20%,35.000000,1.000000,12.000000
50%,141.000000,2.000000,50.000000
80%,276.000000,7.000000,205.000000
96%,347.000000,29.000000,869.000000
max,365.000000,2356.000000,37020.000000


## Monetary | frequency | recency

Распределяю данные по категориям.

1 категория - 1000 - 29 - 35 (~96%)

2 категория - 205 - 7 - 141 (80%)

3 категория - 50 - 2 - 276 (50%)

4 категория - 12 - 1 - 347 (20%)

In [26]:
q = rfm_stats.copy()

In [27]:
def Mon(x):
    if x > 1000:
        return 1
    elif x > 205:
        return 2
    elif x > 50:
        return 3
    else:
        return 4
    
def Freq(x):
    if x > 29:
        return 1
    elif x > 7:
        return 2
    elif x > 2:
        return 3
    else:
        return 4

def Rec(x):
    if x > 347:
        return 4
    elif x > 276:
        return 3
    elif x > 141:
        return 2
    else:
        return 1

In [29]:
q['Mrank'] = q.monetary.apply(Mon)

In [30]:
q['Frank'] = q.frequency.apply(Freq)

In [31]:
q['Rrank'] = q.recency.apply(Rec)

In [32]:
q.sort_values(['Mrank', 'Frank', 'Rrank'])

,recency,frequency,monetary,Mrank,Frank,Rrank
user_id,,,,,,
00bc44a61dafb62e5c2b54200573796e,69,99,1417.0,1,1,1
00e17c2f971bd28c7836d7352d383edd,2,63,1959.0,1,1,1
00f3e86e4a9162e900e8dc5afeca0b94,54,36,1031.0,1,1,1
019112da0af39f39823cb27b03219d86,37,32,1543.0,1,1,1
01bb3fe2b7ff05ec3210f938e5258bcb,9,89,2892.0,1,1,1
...,...,...,...,...,...,...
fed97e44daeaa8bbd933035495bc567a,358,1,10.0,4,4,4
ff3784e4baa068e3232387b358aa98e5,362,1,10.0,4,4,4
ff7adfd79ff3bb28746caa61ca652c01,362,1,17.0,4,4,4


In [ ]:
q[q.Frank == 3]

In [ ]:
q[(q.Rrank == 4) & (q.Frank == 2)]

#### Ушедшими, буду считать людей с Rrank == 4

### Главные задачи
### 1) Уменьшить отток пользователей 
### 2) Увеличить частоту контакта

In [ ]:
q[q.Rrank == 3]

## Часто переходящие пользователи на грани ухода

Необходимо сделать рассылку на новый товар/дать скидку/новое предложение

In [ ]:
q[(q.Rrank == 3) & ((q.Frank == 1) | (q.Frank == 2))]

## Ушедшие, но часто переходящие пользователи

Необходимо узнать причину ухода 

In [ ]:
q[(q.Rrank == 4) & ((q.Frank == 1) | (q.Frank == 2))]

## недавно перешедшие по ссылке пользователи

нужно стимулировать новые переходы, предложением дополняющих товаров

In [ ]:
q[((q.Rrank == 2) | (q.Rrank == 1)) & ((q.Frank == 2) | (q.Frank == 1) | (q.Frank == 3))]